# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
# from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [52]:
# Loading the output csv file generated in weatherpy to a dataframe
vacation_df=pd.read_csv("../WeatherPy/Output/city_weather.csv")
vacation_df.iloc[220:235,:]

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
220,220,Mt. Pleasant,US,32.7941,-79.8626,81.00,90,1,1.99,1626229988
221,221,Alofi,NU,-19.0595,-169.9187,82.29,74,75,10.36,1626230246
222,222,Cabo San Lucas,MX,22.8909,-109.9124,87.80,48,75,4.00,1626230247
223,223,Saint Paul Harbor,US,57.7900,-152.4072,54.91,76,90,10.36,1626230247
224,224,Arroyo de la Encomienda,ES,41.6096,-4.7969,56.21,71,0,1.01,1626230247
225,225,Mossamedes,AO,-15.1961,12.1522,62.20,89,57,0.78,1626230034
226,226,Cockburn Town,TC,21.4612,-71.1419,80.64,84,0,16.37,1626230247
227,227,Port Macquarie,AU,-31.4333,152.9167,60.39,51,100,6.26,1626230185
228,228,Vanavara,RU,60.3400,102.2797,72.45,48,100,9.33,1626230248
229,229,Zhireken,RU,52.8249,117.3076,68.14,54,100,13.35,1626230248


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [53]:
# Configuring gmaps with api key
gmaps.configure(api_key='api_keys/g_key')

In [54]:
# locating gmap by latitude and longitude and Humidity as weight
locations = vacation_df[['Latitude', 'Longitude']] 
weights = vacation_df['Humidity']

# plotting base gmap 
fig = gmaps.figure()

# Adding heatmap layer over the base map
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
